This file generates the initial test and training sets randomly, which are now fixed in mocap_labels.py.
This also ensures that at every subject has at least one trial in the training set for walking and running each.

In [10]:
# Add the parent directory to the path so that we can import other modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)



In [11]:
from dataset_utils.mocap_labels import __walk_trials, __run_trials
import random

In [12]:
def gen_test_train_dicts(trial_dict, proportion_train=0.33333):

    # Flatten the dictionary into a list of (subject, trial) pairs
    all_trials = [(subject, trial) for subject, trials in trial_dict.items() for trial in trials]

    n_train = round(len(all_trials) * proportion_train)

    # Start by choosing one random trial for each subject to be in the training set
    train_trials = []
    for subject, trials in trial_dict.items():
        train_trials.append((subject, random.choice(trials)))

    # Remove the selected trials from the list
    for subject, trial in train_trials:
        all_trials.remove((subject, trial))

    if len(train_trials) < n_train:

        # Add more random trials to the training set until we have enough
        additional_train_trials = random.sample(all_trials, n_train - len(train_trials))
        train_trials.extend(additional_train_trials)

    # Rebuild the dictionary with selected trials
    train_data = {}
    for subject, trial in train_trials:
        if subject not in train_data:
            train_data[subject] = []
        train_data[subject].append(trial)

    # Put all the remaining trials into the test data
    test_data = {}
    for subject, trial in all_trials:
        if (subject, trial) not in train_trials:
            if subject not in test_data:
                test_data[subject] = []
            test_data[subject].append(trial)

    return train_data, test_data

In [13]:
walk_train, walk_test = gen_test_train_dicts(__walk_trials, 1/3)
print("__walk_trials_train =", walk_train)
print("__walk_trials_test =", walk_test)

__walk_trials_train = {2: [2], 5: [1], 6: [1], 7: [12, 7, 11], 8: [9], 10: [4], 12: [2, 1]}
__walk_trials_test = {2: [1], 7: [1, 2, 3, 4, 5, 6, 8, 9, 10], 8: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11], 12: [3]}


In [15]:
run_train, run_test = gen_test_train_dicts(__run_trials, 1/3)
print("__run_trials_train =", run_train)
print("__run_trials_test =", run_test)

__run_trials_train = {2: [3], 9: [3, 8, 2, 10], 16: [45, 46], 35: [20, 25]}
__run_trials_test = {9: [1, 4, 5, 6, 7, 9, 11], 16: [35, 36, 55, 56], 35: [17, 18, 19, 21, 22, 23, 24]}
